# An Example of Forecasting / Nowcasting with DLT

In this session, we will explore:

- Orbit Installation
- A forecasting task on iclaims dataset
- A simple DLT model
- DLT model with regression settings
- Disgnoses

For more examples you can find on [Github](https://github.com/uber/Orbit).

In [ ]:
%load_ext lab_black

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import orbit
from orbit.utils.dataset import load_iclaims
from orbit.models.dlt import DLTFull, DLTMAP
from orbit.diagnostics.plot import (
    plot_predicted_data,
    plot_predicted_components,
    plot_posterior_params,
)
from orbit.constants.palette import QualitativePalette
from orbit.diagnostics.metrics import smape
from sklearn.preprocessing import MinMaxScaler

from pylab import rcParams
import warnings

warnings.simplefilter("ignore", category=FutureWarning)

rcParams["figure.figsize"] = 14, 8
plt.style.use("fivethirtyeight")

In [ ]:
def mae(x, y):
    return np.mean(np.abs(x - y))

# US Weekly Initial Claims

The *iclaims* data contains the weekly initial claims for US unemployment benefits against a few related google trend queries (unemploy, filling and job) 
from Jan 2010 - June 2018. This dataset was used in the original Bayesian Structural Time-Series paper [Scott and Varan (2013)](https://people.ischool.berkeley.edu/~hal/Papers/2013/pred-present-with-bsts.pdf) as well.

Number of claims are obtained from [Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/ICNSA) while regressors such as google queries are obtained through [Google Trends API](https://trends.google.com/trends/?geo=US).

In order to use this data to nowcast the US unemployment claims considering the impact of COVID-19, we extended the dataset to Jan 2021 and added the [S&P 500 (^GSPC)](https://finance.yahoo.com/quote/%5EGSPC/history?period1=1264032000&period2=1611187200&interval=1wk&filter=history&frequency=1wk&includeAdjustedClose=true) and [VIX](https://finance.yahoo.com/quote/%5EVIX/history?p=%5EVIX) Index historical data for the same period.

**Note:** The data is standardized and log-transformed for the model fitting purpose.

In [ ]:
# load data
df = load_iclaims(end_date="2021-01-03")
df = df[["week", "claims", "trend.unemploy", "trend.job", "sp500", "vix"]]
date_col = "week"
response_col = "claims"
df.dtypes

To introduce forecastability, we use one week lag of data for the regressors.

In [ ]:
df[["trend.unemploy", "trend.job", "sp500", "vix"]] = df[
    ["trend.unemploy", "trend.job", "sp500", "vix"]
].shift(1)
df = df[1:].reset_index(drop=True)

In [ ]:
df.head()

We can see from the charts below, there are seasonlity, trend, and as well as a huge changpoint due the impact of COVID-19.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(20, 8))
axs[0, 0].plot(df["week"], df["claims"])
axs[0, 0].set_title("Unemployment Claims")
axs[0, 1].plot(df["week"], df["trend.unemploy"], "tab:orange")
axs[0, 1].set_title("Google trend - unemploy")
axs[1, 0].plot(df["week"], df["vix"], "tab:green")
axs[1, 0].set_title("VIX")
axs[1, 1].plot(df["week"], df["sp500"], "tab:red")
axs[1, 1].set_title("S&P500")

## Train / Test Split

To make Bayesian priors comparable across regressors, we need some transformation across regressors.

In [ ]:
# using relatively updated data
df = df[df["week"] > "2018-01-01"].reset_index(drop=True)
test_size = 12
train_df = df[:-test_size].reset_index(drop=True)
test_df = df[-test_size:].reset_index(drop=True)

scaler = MinMaxScaler()
train_df[
    ["claims", "trend.unemploy", "trend.job", "sp500", "vix"]
] = scaler.fit_transform(
    train_df[["claims", "trend.unemploy", "trend.job", "sp500", "vix"]]
)
test_df[["claims", "trend.unemploy", "trend.job", "sp500", "vix"]] = scaler.transform(
    test_df[["claims", "trend.unemploy", "trend.job", "sp500", "vix"]]
)

# Simple DLT Model

In [ ]:
%%time
dlt = DLTFull(
    response_col=response_col,
    date_col=date_col,
    seasonality=52,
    seed=8888,
    num_warmup=400,
    num_sample=100,
)

dlt.fit(df=train_df)
predicted_df = dlt.predict(df=test_df)

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                        date_col=date_col, actual_col=response_col, test_actual_df=test_df,
                        title='DLT with Linear Global Trend')

# DLT With Regression

The regressor columns can be supplied via argument `regressor_col`.  Recall the regression formula in **DLT**:

$$
\hat{y}_t =\mu_t + s_t + r_t \\
r_t = \sum_{j}\beta_j x_{jt} \\
\beta_j ~\sim \mathcal{N}(\mu_j, \sigma_j^2)
$$

Let's use the default where $\mu_j = 0$ and $\sigma_j = 1$.  In addition, we can set a *sign* constraint for each coefficient $\beta_j$.  This is can be done by supplying the `regressor_sign` as a list where elements are in one of followings:

* '=': $\beta_j ~\sim \mathcal{N}(0, \sigma_j^2)$  i.e. $\beta_j \in (-\inf, \inf)$
* '+': $\beta_j ~\sim \mathcal{N}^+(0, \sigma_j^2)$  i.e. $\beta_j \in [0, \inf)$
* '-': $\beta_j ~\sim \mathcal{N}^-(0, \sigma_j^2)$  i.e. $\beta_j \in (-\inf, 0]$

Based on some intuition, it's reasonable to assume search terms such as "unemployment", "filling" and **VIX** index to be positively correlated and stock index such as **SP500** to be negatively correlated to the outcome.  Then we will leave whatever unsured as a regular regressor.

## Regular Regression

In [ ]:
dlt_reg = DLTFull(
    response_col=response_col, 
    date_col=date_col,
    regressor_col=['trend.unemploy', 'trend.job', 'sp500', 'vix'],
    seasonality=52,
    num_warmup=4000,
    num_sample=1000,
)

dlt_reg.fit(df=train_df)
predicted_df_reg = dlt_reg.predict(test_df, decompose=True)

_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df_reg, 
                        date_col=date_col, actual_col=response_col, test_actual_df=test_df,
                        title='DLT with Regular Regresion')

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df_reg, 
                        date_col=date_col, actual_col=response_col, test_actual_df=test_df,
                        title='DLT with Regular Regresion')

The estimated regressor coefficients can be retrieved via `.get_regression_coefs()`.

In [ ]:
dlt_reg.get_regression_coefs()

### Diagnoses
#### Decomposition 

`plot_predicted_components` is the utility to plot each component separately. This is useful when one wants to look into the model prediction results and inspect each component separately.

In [ ]:
_ = plot_predicted_components(predicted_df_reg, date_col, 
                              plot_components=['prediction', 'trend', 'seasonality', 'regression'])

#### Posterior Diagnostic Visualizations

`plot_posterior_params` is the main utility for different kinds of diagnostic plots.

##### Trace plot

Trace plot shows the iterations of each paramter over the Markov chian sampling process. Trace plots provide an important tool for assessing mixing of a chain.

In [ ]:
_ = plot_posterior_params(dlt_reg, kind='trace',
                          incl_trend_params=False, incl_smooth_params=False)

##### Density/Histogram

By setting `kind = 'density'`, we get posterior paramter density plot. It shows the mean, median and confidence Interval (95% by default) of various paramter posterior samples. One can specify a path string (e.g., './density.png') to save the chart.

In [ ]:
_ = plot_posterior_params(dlt_reg, kind='density',
                          incl_trend_params=False, incl_smooth_params=False)

##### Pair Plot

By setting `kind = 'pair'`, it will generates a series of pair plots, which depict the relationship between every two parameters.

In [ ]:
_ = plot_posterior_params(dlt_reg, kind='pair', pair_type='reg', 
                          incl_trend_params=False, incl_smooth_params=False)

## Regression with Informative Priors / Regularized Priors

Assuming users obtain further knowledge on some of the regressors, they could use informative priors ($\mu$, $\sigma$) by replacing the defaults. This can be done via the arguments `regressor_beta_prior` and `regressor_sigma_prior`. These two lists should be of the same lenght as `regressor_col`.

In [ ]:
dlt_reg_adjust = DLTFull(
    response_col=response_col,
    date_col=date_col,
    regressor_col=['trend.unemploy', 'trend.job', 'sp500','vix'],
    # regressor_sign=['+','=','-','+'],
    # regressor_beta_prior=['0.3, 0.01, -0.09, -0.016],
    regressor_sigma_prior=[0.1] * 4,
    seasonality=52,
    seed=8888,
    num_warmup=4000,
    num_sample=1000,
)
dlt_reg_adjust.fit(df=train_df)
predicted_df_reg_adjust = dlt_reg_adjust.predict(test_df, decompose=True)

_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df_reg_adjust, 
                        date_col=date_col, actual_col=response_col, test_actual_df=test_df,
                        title='DLT with Regresion of Informative Priors')

In [ ]:
_ = plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df_reg_adjust, 
                        date_col=date_col, actual_col=response_col, test_actual_df=test_df,
                        title='DLT with Regresion of Informative Priors')

In [ ]:
dlt_reg_adjust.get_regression_coefs()

In [ ]:
naive_mae = mae(predicted_df['prediction'].values, test_df['claims'].values)
reg_mae = mae(predicted_df_reg['prediction'].values, test_df['claims'].values)
reg_adjust_mae = mae(predicted_df_reg_adjust['prediction'].values, test_df['claims'].values)

print('Naive Model: {:.3f}\nRegression Model: {:.3f}\nRefined Regression Model: {:.3f}'.format(
    naive_mae, reg_mae, reg_adjust_mae
))